In [18]:
import cv2
import filters
from manager import WindowManager, CaptureManager
import rects
from tracker import facetracker

class click(object):
    
    def __init__(self):
        self._windowmanager=WindowManager('click',self.onkeypress)
        
        self._capturemanager=CaptureManager(cv2.VideoCapture(0),self._windowmanager,True)
        
        self._facetracker=facetracker()
        self.shoulddrawdebugrects=False
        self._curvefilter=filters.bgrportacurvefilter()
        
    def run(self):
        """run the main loop"""
        
        self._windowmanager.createwindow()
        
        while self._windowmanager.iswindowcreated:
            
            self._capturemanager.enterFrame()
            frame=self._capturemanager.frame
            
            #todo : to track faces and swap in one camera feed
            
            self._facetracker.update(frame)
            faces=self._facetracker.faces
            rects.swaprects(frame,frame,
                           [face.facerect for face in faces])
            
            #todo to add filters
            
            filters.strokeedge(frame,frame)
            self._curvefilter.apply(frame,frame)
            
            
            if self._shoulddrawdebugrects:
                self._facetracker.drawdebugrects(frame)
                
            
            
            
            
            self._capturemanager.exitframe()
            self._windowmanager.processevents()
            
    def onkeypress(self,keycode):
        
        """handle a keypress
            
            space-> take a screenshot
            
            tab-> start/stop recording a screenshot
            
            x-> start/stop drawing debug rectangles around faces
            
            escape-> quit
            """
        if keycode == 32:# space
            self._capturemanager.writeimage('screenshot.png')
            
        elif keycode == 9: #tab
            if not self._capturemanager.iswritingvideo:
                self._capturemanager.startwritingvideo('screenshot.avi',360)
                
            else:
                self._capturemanager.stopwritingvideo()
        elif keycode==120: #x
            self._shoulddrawdebugrects= not self.shoulddrawdebugrects
        
        elif keycode==27: #esc
            self._windowmanager.destroywindow()
            
            
            
class clickdouble(click):
    
    def __init__(self):
        click.__init__(self)
        self._hiddencapturemanager=capturemanager(cv2.VideoCapture(1))
            
    def run(self):
        """run the main loop"""
        
        self._windowmanager.createwindow()
        while self._windowmanager.enterframe():
            
            self._capturemanager.enterframe()
            self._hiddencapturemanager.enterframe()
            
            frame=self._capturemanager.frame
            hiddenframe=self._hiddencapturemanager.frame
            
            
            self._facetracker.update(hiddenframe)
            
            hiddenfaces=self._facetracker.faces
            
            self._facetracker.update(frame)
            faces=self._facetracker.faces
            
            i=0
            while i<len(faces) and i < len(hiddenfaces):
                
                rects.copyrect(hiddenframe,frame,hiddenfaces[i].facerect,
                              faces[i].facerect)
                i+=1
                
                
            filters.strokeedges(frame,frame)
            
            self._curvefilter.apply(frame,frame)
            
            if self._shoulddrawdebugrects:
                self._facetracker.drawdebugrects(frame)
                
            self._capturemanager.exitframe()
            self.hiddencapturemanager.exitframe()
            self._windowmanager.processEvents()
            
            
if __name__=="__main__":
    click().run()#uncomment for single camera
    #clickdouble().run()
    
        
    

SyntaxError: invalid syntax (<ipython-input-18-746ec7fd4769>, line 3)

# manager.py


In [ ]:



import cv2
import numpy as np
import time



class CaptureManager(object):
    
    
    
    def __init__(self,capture,previewWindowManager=None,
                 shouldMirrorPreview=False):
       
        self.previewWindowManager=previewWindowManager
        self.shouldMirrorPreview=shouldMirrorPreview
        
        self._capture=capture
        self._channel=0;
        self._enteredframe=False
        self._frame=None
        
        self._imagefilename=None
        self._videofilename=None
        self._videoencoding=0
        self._videowriter=None
        
        self._starttime=None
        self._frameselapsed=long(0)
        self._fpsestimate=None
        
    @property   
    def channel(self):
        return self._channel
    
    
    
    @channel.setter
    def channel(self,value):
        if self._channel != value:
            self._channel=value
            self._frame=None
            
    @property
    def frame(self):
        if self._enteredframe and self._frame is None:
            _,self._frame=self._capture.retrieve(channel=self.channel)
        
        return self._frame
    
    @property
    def isWritingImage(self):
        return self._imagefilename is not None
    
    @property
    def isWritingVideo(self):
        return self._videofilename is not None
    
    
    def enterframe(self):
        #capture the next frame if any
        
        #but check if there if any other frame
        
        
        if self._capture is not None:
            self._enteredframe=self._capture.grab()
    
    
    
    def exitframe(self):
        
        #to check whether any grabbed frame is retrievable.
        if self.frame is not None:
            self._enteredframe=False
            return
        
        #update the fps estimate and related variables
        
        if self._frameselasped == 0:
            self._starttime=time.time()
        
        else:
            timeelasped=time.time()-self._starttime
            self._fpsestimate=self._frameselapsed/timeelasped
            
        self._frameselapsed+=1
        
        
        
        
        #draw a window ,if any.
        
        
        if self.previewWindowManager is not None:
            if self.shouldMirrorPreview:
                
                mirroredframe=np.fliplr(self._frame).copy()
                self.previewWindowManager.show(self.frame)
                
                
        #write image file if any is present
        
        if self.iswritingimage:
            cv2.imwriter(self.imagefilename,self.frame)
            self._imagefilename=None
            
        #write  to the video file
        
        self.writevideoframe()
        
        
        
        #release the frame
        
        self._frame=None
        self._enteredframe=False
        
                
    def writeimage(self,filename):
        """writing next exited frame to an image file"""
        
        self.imagefilename=filename
        
    
    def startwritingvideo(self,filename,encoding=360):
        
        """start writing exited filename to video filename"""
        
        self._videofilename=filename
        self._videoencoding=encoding
        
        
    def stopwritingvideo(self):
        """stop writing exited frame to a video file."""
        
        self._videofilename=None
        self._videoencoding=None
        self._videowriter=None
        
        
    def writevideoframe(self):
          
        if not self.iswritingvideo:
            return
        
        if self._videowriter is None:
            
            fps=self._capture.get(cv2.cv.CV_CAP_PROP_FPS)
            
            if fps==0.0:
                #the capture's fps is unknown so use an estimate.
                if self._frameselapsed < 20:
                    """wait until more frame elapsed so that the estimate is more stable"""
                    return
                else:
                    fps=self._fpsestimate
            
            size=(int(self._capture.get(cv2.cv.CV_CAP_PROP_FRAME_WIDTH)),
                 int(self._capture.get(cv2.cv.CV_CAP_PROP_FRAME_HEIGHT)))
            
            self._videowriter=cv2.VideoWriter(self._videofilename,self._videoencoding ,fps,size)
        
                  
                  
        self._videowriter.write(self._frame)      
    
                  
                  
                  

                
                
                  
class WindowManager(object):
    
                  
    
                  
    def __init__(self,windowname,keypresscallback=None):
        self.keypresscallback=keypresscallback
        self._windowname=windowname
        self._iswindowcreated=False
                  
            
                  
            
    def iswindowcreated(self):
                  
        return self._iswindowcreated
                  
    def createwindow(self):
        cv2.namedWindow(self._windowname)
        self._iswindowcreated=True
                  
            
    def show(self,frame):
        cv2.imshow(self._windowname,frame)
                  
    def destroywindow(self):
        cv2.destroyWindow(self._windowname)
        self._iswindowcreated=False
                  
    def processevents(self):
        keycode=cv2.waitKey(1)
                 
        if self.keypresscallback is not None and keycode != -1:
                  
                  keycode&= 0xFF
                  self.keypresscallback(keycode)

                   
                  
            
                  
                  

    
    
    
    
    
    
    
    

# filters.py

In [ ]:
import cv2
import numpy as np
import util as ut




class vfuncfilter(object):
    
    """a filter  that applies a function to V(or all of bgr)"""
    
    def __init__(self,vfunc=None,dtype=np.uint8):
        length=np.iinfo(dtype).max+1
        self._vlookuparray=ut.createlookuparray(vfunc,length)
        
    
    def apply(self,src,dst):
        """apply the filter with a bgr or gray source/destination"""
        
        srcflatview=ut.flatview(src)
        dstflatview=ut.flatview(dst)
        ut.applylookuparray(self._vlookuparray,srcflatview,dstflatview)
        
        


        
class vcurvefilter(vfuncfilter):
    
    
    
    """a filter which apply a curve to v(all bgr)"""
    
    def __init__(self,vpoints,dtype=np.uint8):
        
        vfuncfilter.__init__(self,ut.createcurvefunc(vpoints),dtype)
        
        
    
    

    
class bgrfuncfilter(object):
    """a filter that applies different function to each of bgr."""
    
    def __init__(self,vfunc=None,bfunc=None,gfunc=None,rfunc=None,dtype=np.uint8):
        
        length=np.iinfo(dtype).max+1
        
        self._blookuparray=ut.createlookuparray(
            ut.createcompositefunc(bfunc,vfunc),length)
        self._glookuparray=ut.createlookuparray(
            ut.createcompositefunc(gfunc,vfunc),length)
        self._rlookuparray=ut.createlookuparray(
            ut.createcompositefunc(rfunc,vfunc),length)
        
    
    
    def apply(self,src,dst):
        """apply the filters with a bgr source/destination"""
        
        b,g,r=cv2.split(src)
        
        ut.applylookuparray(self._blookuparray,b,b)
        
        ut.applylookuparray(self._blookuparray,b,b)
        
        ut.applylookuparray(self._blookuparray,b,b)
        
        cv2.merge([b,g,r],dst)
        
        
        
        
class bgrcurvefilter(bgrfuncfilter):
    """a filter that applies different curves to each of bgr"""
    
    def __init__(self,
                 vpoints=None,
                 bpoints=None,
                 gpoints=None,
                 rpoints=None,
                 dtype=np.uint8
                ):
        
        bgrfuncfilter.__init__(self,
                              ut.createcurvefunc(vpoints),
                              ut.createcurvefunc(vpoints),
                              ut.createcurvefunc(vpoints),
                              ut.createcurvefunc(vpoints),
                               dtype
                              )
            
            
            
class bgrPortraCurveFilter(bgrCurveFilter):   
    """A filter that applies Portra-like curves to bgr."""        
    
    def __init__(self, dtype = numpy.uint8):        
        
        bgrCurveFilter.__init__(            self,           
                                vPoints = [(0,0),(23,20),(157,173),(255,255)],
                                bPoints = [(0,0),(41,46),(231,228),(255,255)],
                                gPoints = [(0,0),(52,47),(189,196),(255,255)],
                                rPoints = [(0,0),(69,69),(213,218),(255,255)],
                                dtype = dtype) 
        
        
        
class bgrProviaCurveFilter(bgrCurveFilter):    
    """A filter that applies Provia-like curves to bgr."""
    def __init__(self, dtype = numpy.uint8):        
        
        bgrCurveFilter.__init__(self,            
                                bPoints = [(0,0),(35,25),(205,227),(255,255)],
                                gPoints = [(0,0),(27,21),(196,207),(255,255)],
                                rPoints = [(0,0),(59,54),(202,210),(255,255)],
                                dtype = dtype)
        
        
        
class bgrVelviaCurveFilter(bgrCurveFilter):
    """A filter that applies Velvia-like curves to bgr."""
    
    def __init__(self, dtype = numpy.uint8):        
        
        bgrCurveFilter.__init__(            self,
                                vPoints = [(0,0),(128,118),(221,215),(255,255)],
                                bPoints = [(0,0),(25,21),(122,153),(165,206),(255,255)],
                                gPoints = [(0,0),(25,21),(95,102),(181,208),(255,255)],
                                rPoints = [(0,0),(41,28),(183,209),(255,255)],
                                dtype = dtype)        
        
        
class bgrCrossProcessCurveFilter(bgrCurveFilter):
    """A filter that applies cross-process-like curves to bgr."""
    
    def __init__(self, dtype = numpy.uint8):        
        
        bgrCurveFilter.__init__(            self,
                                bPoints = [(0,20),(255,235)],
                                gPoints = [(0,0),(56,39),(208,226),(255,255)],
                                rPoints = [(0,0),(56,22),(211,255),(255,255)],
                                dtype = dtype)
        


        
        
def strokeedges(src,dst,blurksize=7,edgeksize=5):
    
    if blurksize>=3:
        
        blursrc=cv2.medianBlur(src,blurksize)
        graysrc=cv2.cvtColor(blursrc,cv2.COLOR_bgr2GRAT)
    
    else:
        
        graysrc=cv2.cvtColor(src,cv2.COLOR_bgr2GRAY)
        
    cv2.laplacian(graysrc,cv2.cv.CV_8U,graysrc,ksize=edgeksize)
    
    normalizedinversealpha=(1.0/255)*(255-graysrc)
    channels=cv2.split(src)
    for channel in channels:
        
        channel[:]=channel*normalizedinversealpha
        
    cv2.merge(channels,dst)

    
    
class vconvolutionfilter(object):
    """a filter that applies a convolution to v(or all of bgr)
    
    """
    
    def __init__(self,kernel):
        
        self._kernel=kernel
        
    def apply(self,src,dst):
        
        """apply the filter with a bgr or gray source/destination"""
        
        cv2.filter2D(src,-1,self.kernel,dst)
        
        
class sharpenfilter(vconvolutionfilter):
    """a sharpen filter with a 1-pixel radius"""
    
    def __init__(self):
        
        kernel=np.array([[-1, -1, -1],
                         [-1,  9, -1],
                         [-1, -1, -1]])
        vconvolutionfilter.__init__(self,kernel)
        
        
        

        
class findedgefilter(vconvolutionfilter):
    
    """
    an edge finding filter with a pixel radius"""
    
    def __init__(self):
        
        kernel=np.array([[-1 , -1,  -1],
                          [-1  ,8   -1],
                          [-1 ,-1,  -1]])
        
        vconvolutionfilter.__init__(self.kernel)
        
        
        
class blurfilter(vconvolutionfilter):
    
    """a blur filter with a 2 pixel radius"""
        
    def __init__(self):
            kernel= np.array([[0.04, 0.04, 0.04, 0.04, 0.04],
                                 [0.04, 0.04, 0.04, 0.04, 0.04],
                                 [0.04, 0.04, 0.04, 0.04, 0.04],
                                 [0.04, 0.04, 0.04, 0.04, 0.04],
                                 [0.04, 0.04, 0.04, 0.04, 0.04]])
            
            vconvolutionfilter.__init__(self,kernel)
            

            
            
class embossfilter(vconvolutionfilter):
    
    """an emboss filter with a 1 pixel radius"""
    
    def __init__(self):
        kernel=np.array([[-2, -1, 0],
                         [-1,  1, 1],
                         [ 0,  1, 2]]) 
        
        
        vconvolutionfilter.__init__(self,kernel)

# util.py


In [ ]:
import cv2
import numpy as np
import scipy.interpolate




def createcurvefunc(points):
    """return a function dervied from control points"""
    
    if points is None:
        return None
    numpoints=len(points)
    
    if numpoints <2:
        return None
    
    xs,ys=zip(*points)
    if numpoints<3:
        kind="linear"
        
    else:
        kind="cubic"
        
    return scipy.interpolate.interp1d(xs,ys,kind,bounds_error=False)


def createlookuparray(func,length=256):
    
    """return a lookup for whole-number input to a function
    the look up array value are clamped to [0,length-1]
    
    """
    if func is None:
        return None
    
    lookuparray=np.empty(length)
    
    i=0
    while i<length:
        func_i=func(i)
        lookuparray[i]=min(max(0,func_i),length-1)
        i+=1
    return lookuparray


def applylookuparray(lookuparray,src,dst):
    """map a source to a destination using a lookup"""
    
    if lookuparray is None:
        return 
    dst[:]=lookuparray[src]
    
    
def createcompositionfunc(func0,func1):
    """return a composition of two function"""
    
    if func0 is None:
        return func1
    if func1 is None:
        return func0
    return lambda x:func0(func1(x))


def createflatview(array):
    """return a 1d view of any array of any dimensionality"""
    
    flatview=array.view()
    flatview.shape=array.size
    return flatview

def isgray(image):
    """return true if image is color """
    return image.ndim<3

def widthheigthdividedby(image,divisor):
    """return an image dimension,divided by a value."""
    
    h,w=image.shape[:2]
    return (w/divisor,h/divisor)



# rects.py

In [ ]:
import cv2


def outlinerect(image,rect,color):
    if rect in None:
        return
    x,y,w,h=rect
    cv2.rectangle(image,(x,y),(x+w,y+h),color)
    
    

    
def copyrect(src,dst,srcrect,dstrect,
             interpolation=cv2.INTER_LINEAR):
    """COPY part of the source part of the destination."""
    
    x0,y0,w0,h0=srcrect
    x1,y1,w1,h1=dstrect
    
    #resize the contents of the source sub-rectangle.
    #put the results in destination sub-rectangle.
    
    dst[y1:y1+h1,x1:x1+w1]=cv2.resize(src[y0:y0+h0,x0:x0+w0],(w1,h1),interpolation=interpolation)
    
def swaprect(src,dst,rects,interpolation=cv2.INTER_LINEAR):
    
    """COPY THE SOURCE WITH TWO OR MORE SUB RECTANGLE SWAPPED"""
    
    if dist is not src:
        dst[:]=src
    
    numrects=len(rects)
    
    if numrects<2:
        #copy the contents of last rectangle into temporay storage
        
        x,y,w,h=rects[numrects-1]
        temp=src[y:y+h,x:x+w].copy()
        
        #copy the content of each rectangle into the next
        
        i=numrects-2
        
        while i>=0:
            copyrect(src,dst,rects[i+1],interpolation)
            i-=1
            
        #copy the temporarily stored content into the first rectangle
        
        copyrect(temp,
                 dst,
                 (0,0,w,h),
                 rect[0],
                interpolation)
    
    
    

# tracker.py


In [ ]:
import cv2
import rects
import util


class face(object):
    """
    data on facial features"""
    def __init__(self):
        self.facerect=None
        self.rigtheyerect=None
        self.lefteyerect=None
        self.noserect=None
        self.mouthrect=None
        
class facetracker(object):
    """a tracker for facial features:face,eyes,nose,mouth"""
    
    def __init__(self,
                 scaleFactor=1.2,
                 minNeighbors=2,
                flags=cv2.cv.CV_HAAR_SCALE_IMAGE):
        
        
        self.scalefactor=scaleFactor
        self.minneighbors=minNeighbors
        self.flags=flags
        
        self._faces=[]
        
        self._faceclassifier=cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
        self._eyeclassifier=cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
        self._noseclassifier=cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
        self._mouthclassifier=cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
        
        
        
    
    @property
    def faces(self):
        """the tracked facial features"""
        return self._faces
    
    
    def update(self,image):
        """update the tracked facial features"""
        
        self._faces=[]
        
        if util.isgray(image):
            image=cv2.equalizeHist(image)
            
        else:
            
            image=cv2.cvtColor(image,cv2.cv.CV_BGR2GRAY)
            cv2.equalizeHist(image,image)
            
        minsize=util.widthheightdividedby(image,8)
        
        facerects=self._faceclassifier.detectMultiScale(image,
                                                        self.scalefactor,
                                                        self.minneighbors,
                                                        self.flags,
                                                        minsize)
        
        if facerects is not None:
            
            for facerect in facerects:
                face=face()
                
                face.facerect=facerect
                
                
                x,y,w,h=facerect
                
                # Seek an eye in the upper-left part of the face.                
                searchRect = (x+w/7, y, w*2/7, h/2)                
                face.leftEyeRect = self._detectOneObject(                    
                    self._eyeClassifier, image, searchRect, 64)                                
                
                
                
                # Seek an eye in the upper-right part of the face.                
                searchRect = (x+w*4/7, y, w*2/7, h/2)                
                face.rightEyeRect = self._detectOneObject(                    
                    self._eyeClassifier, image, searchRect, 64)                                
                
                
                
                # Seek a nose in the middle part of the face.                
                searchRect = (x+w/4, y+h/4, w/2, h/2)                
                face.noseRect = self._detectOneObject(                    
                    self._noseClassifier, image, searchRect, 32)                                
               
                # Seek a mouth in the lower-middle part of the face.                
                searchRect = (x+w/6, y+h*2/3, w*2/3, h/3)                
                face.mouthRect = self._detectOneObject(                    
                    self._mouthClassifier, image, searchRect, 16)                                
                
                
                
                self._faces.append(face)

        
        
        def _detectoneobject(self,
                             classifier,
                             image,
                             rect,
                             imagesizetominsizeratio):
            
            x ,y ,w ,h=rect
            
            minsize=util.widthheightdividedby(image,
                                               imagesizetominsizeratio)
            
            subimage=image[y:y+h,x:x+w]
            
            subrect=classifier.dectectMultiScale(subimage,
                                                self.scalefactor,
                                                self.minneighbors,
                                                self.flags,
                                                minsize)
            
            if len(subrect)==0:
                return None
            
            subx,suby,subw,subh=subrects[0]
            
            return (x+subx,y+suby,w+subw,h+subh)
        
        
        
        def drawdebugrects(self,image):
            
            """draw rectangle around the tracked facial features."""
            
            if util.isgray(image):
                faceColor = 255            
                leftEyeColor = 255            
                rightEyeColor = 255            
                noseColor = 255           
                mouthColor = 25
                
            else:
                faceColor = (255, 255, 255) # white            
                leftEyeColor = (0, 0, 255) # red           
                rightEyeColor = (0, 255, 255) # yellow            
                noseColor = (0, 255, 0) # green            
                mouthColor = (255, 0, 0) # blue     
                
                
            for face in self.faces:
                
                rects.outlinerect(image,face.facerect,facecolor)
                rects.outlineRect(image, face.leftEyeRect, leftEyeColor)            
                rects.outlineRect(image, face.rightEyeRect,rightEyeColor)            
                rects.outlineRect(image, face.noseRect, noseColor)            
                rects.outlineRect(image, face.mouthRect, mouthColor)